In [1]:
from tqdm.notebook import tqdm
import pandas as pd
pd.set_option("display.max_colwidth", None)
import json
from sentence_transformers import SentenceTransformer, util

In [2]:
df = pd.read_csv("../data/our_valid.csv", keep_default_na=False)

In [3]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [8]:
def get_paraphrase_score(argument, keypoint):
    paraphrases = util.paraphrase_mining(model, [argument, keypoint])
    score, i, j = paraphrases[0]
    return score

In [9]:
arg = """school uniforms cut down on bulling and keep everyone the same."""
kp = """School uniform reduces bullying"""

In [15]:
arg= """austerity is the best way to get the economy back on track"""
kp  = """Austerity regime is essential in economically troubled times"""

In [16]:
get_paraphrase_score(arg, kp)

0.5407996773719788

In [17]:
def match_argument_with_keypoints(result, kp_dict, arg_dict):
    for arg_id, arg in arg_dict.items():
        result[arg_id] = {}
        for kp_id, kp in kp_dict.items():
            result[arg_id][kp_id] = get_paraphrase_score(arg, kp)
    return result

In [18]:
valid_df = df

In [19]:
argument_keypoints = {}
for topic in tqdm(valid_df.topic.unique()):
    for stance in [-1, 1]:
        topic_keypoint_ids = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['key_point_id'].tolist()
        topic_keypoints = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['key_point'].tolist()
        topic_kp_dict = dict(zip(topic_keypoint_ids, topic_keypoints))
        
        topic_argument_ids = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['arg_id'].tolist()
        topic_arguments = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['argument'].tolist()
        topic_arg_dict= dict(zip(topic_argument_ids, topic_arguments))
        # match 
        argument_keypoints = match_argument_with_keypoints(argument_keypoints, topic_kp_dict, topic_arg_dict)

In [20]:
len(argument_keypoints)

653

In [21]:
json.dump(argument_keypoints, open("paraphrase_detection_our_valid_predictions.json", "w",encoding='utf-8'))

In [22]:
! python3 ../src-py/track_1_kp_matching.py ../data paraphrase_detection_our_valid_predictions.json our_valid

mAP strict= 0.7114848640492888 ; mAP relaxed = 0.878784591933353
